# Cuaderno 26: Formulación de planos cortantes para el ATSP
## Implementación con planos cortantes

$\newcommand{\card}[1]{\left| #1 \right|}$
$\newcommand{\tabulatedset}[1]{\left\{ #1 \right\}}$
$\newcommand{\ZZ}{\mathbb{Z}}$
$\newcommand{\RR}{\mathbb{R}}$

Dados: 
* un grafo dirigido **completo** $D=(V,A)$; y,
* un vector $c \in \ZZ^{A}$ de costos asociados a los arcos de $D$.

El *problema del agente viajero asimétrico (Asymmetric Traveling Salesman Problem, ATSP)* consiste en encontrar un circuito dirigido que visite **todos** los nodos de $G$ y que tenga el menor costo posible.

Vamos a considerar la formulación como programa lineal entero para el ATSP que emplea desigualdades de cortes para eliminar subciclos (ver Cuaderno 15):

\begin{align*}
\min &\sum_{(i,j) \in A} c_{ij} x_{ij}\\ 
& \mbox{s.r.}\\
&\sum_{(i,j) \in \delta^{+}(i) } x_{ij} = 1, \quad \forall i \in V,\\
&\sum_{(j,i) \in \delta^{-}(i) } x_{ji} = 1, \quad \forall i \in V,\\
&\sum_{ij \in \delta^{+}(W)} x_{ij} \geq 1, \quad \forall W \subset V, \emptyset \neq W \neq V,\\
& x_{ij} \in \{0, 1\}, \quad \forall (i,j) \in A,
\end{align*}
donde $\delta^{+}(W) := \{ (i,j) \in A \, : \, i \in W, j \not\in W\}$.

El problema de esta formulación es la gran cantidad de desigualdades de corte requeridas. Sin embargo, este modelo puede resolverse eficientemente utilizando un **algoritmo de planos cortantes**, el mismo que consiste en generar dinámicamente las desigualdades de corte, conforme sean requeridas en la solución del modelo.

En este cuaderno vamos a revisar cómo implementar un algoritmo de planos cortantes empleando la interfaz Python de Gurobi.



Definimos primero los datos. Usaremos la función `randint` del módulo random para generar valores aleatorios en el rango {0,..,100} para las coordenadas de cada nodo. Los costos de los arcos serán iguales a las distancias euclideanas entre sus nodos extremos. 

In [ ]:
import gurobipy as gp
from gurobipy import GRB
import random as rm
import math 

# iniciar generador de numeros aleatorios
rm.seed(0)

# número de nodos del grafo
n = 20

# conjunto de nodos del grafo
V = gp.tuplelist(range(1,n+1))

# posiciones de los nodos en un plano euclideano entre (0,0) y (100, 100)
coordx={i : rm.randint(0,100) for i in V}
coordy={i : rm.randint(0,100) for i in V}

# los costos son las distancias eculideanas
c = gp.tupledict({
    (i,j) : math.sqrt((coordx[i] - coordx[j])**2 + (coordy[i] - coordy[j])**2)
    for i in V for j in V if i!=j
})
A = c.keys()


Emplearemos el módulo `matplotlib` para graficar el tour de la solución. Definiremos para ello la función `dibujarTour` que recibe tres argumentos: una lista `coordx` con las coordenadas horizontales de los nodos, una lista `coordy` con las coordenadas verticales y un vector `tour` con una permutación de los nodos indicando el orden de visita en la solución.

In [ ]:
import matplotlib.pyplot as plt
import random

def dibujarTour(coordx, coordy, tour):
    Tx = [coordx[i] for i in tour]
    Ty = [coordy[i] for i in tour]
    plt.plot(Tx[:-1], Ty[:-1], 'ro')
    for i in range(len(tour)-1):
        s='{}'.format(tour[i])
        plt.text(Tx[i],Ty[i]+1,s)
        plt.arrow(Tx[i], Ty[i], Tx[i+1]-Tx[i], Ty[i+1]-Ty[i], color='blue', 
                  length_includes_head=True, width=0.1, head_width=2)
    plt.show()


Definimos ahora el objeto modelo, las variables y la función objetivo. 

Definimos las variables miembro `_x`, `_V` y `_A` dentro del objeto modelo para tener acceso a las variables de selección de arcos, la lista de nodos y la lista de arcos desde la función que se encargará de generar las restricciones de corte. 

In [ ]:
# crear el objeto modelo
m = gp.Model('atsp-corte-lazy')

# crear las variables
x = m.addVars(A, name="x", vtype=GRB.BINARY)

# crear la función objetivo
m.setObjective(x.prod(c,'*'), GRB.MINIMIZE)

# copiar variables en el objeto modelo para tener acceso a x, V y A
m._x = x
m._V = V
m._A = A

Añadimos las restricciones de grado. 

In [ ]:
# restricciones de grado saliente
m.addConstrs((x.sum(i,'*') == 1 for i in V), "g_saliente")

# restricciones de grado entrante
m.addConstrs((x.sum('*', i) == 1 for i in V), "g_entrante")



Definimos una función callback para separar dinámicamente las restricciones de corte. Esta función es invocada automáticamente por Gurobi cada vez que el solver encuentra una nueva solución entera que satisface las restricciones de grado (`where == GRB.Callback.MIPSOL`). 

La función implementa un algoritmo de marcaje para determinar el conjunto W de todos los nodos que pueden ser alcanzados desde el nodo 1. Si este conjunto es distinto de V, el mismo está asociado a una desigualdad de corte violada por `x`. 

Un algoritmo con estas características se conoce en general como un **algoritmo de separación**: se encarga de verificar que una solución satisfaga todas las desigualdades de una familia o encontrar una desigualdad que sea violada por la solución.

Notar que para acceder a las variables de selección de arcos, la lista de nodos y la lista de arcos, es necesario emplear las variables miembro `_x`, `_V`y `_A` de la clase modelo.

In [ ]:
# función callback para separar desigualdades de corte
def mycallback(model, where):
    # esta función se activará cuando se encuentre una nueva solución entera
    if where == GRB.Callback.MIPSOL:
        # recuperar los valores de la solución actual
        vx = model.cbGetSolution(model._x)
        # determinar los arcos seleccionados en la solución
        L = gp.tuplelist([(i,j) for (i,j) in model._A if vx[i,j]>=0.9])
        # construir la lista W de nodos que pueden ser alcanzados desde 1
        W = [1]
        i = 1
        while True :
            # seleccionar el único arco saliente de i en L
            a = L.select(i,'*')[0]
            L.remove(a)
            if a[1]==1: 
                break
            W.append(a[1])
            i = a[1]
        # Si W!=V, agregar la desigualdad de corte asociada a W
        if len(W)!=len(model._V):
            Wc = [i for i in model._V if i not in W]
            model.cbLazy(model._x.sum(W, Wc) >= 1)


Escribimos el modelo a un archivo de texto:

In [ ]:
# escribir el modelo a un archivo
m.write('atsp-corte-lazy.lp')


Para poder utilizar restricciones tipo *lazy* en un modelo, debe fijarse el parámetro `LazyConstraints` al valor de 1.

In [ ]:
# configurar Gurobi para usar restricciones lazy
m.Params.LazyConstraints = 1


Fijamos un límite para el tiempo de cálculo y una tolerancia para la brecha de optimalidad:

In [ ]:
# terminar al alcanzar un Gap del 5%
m.Params.MIPGap = 0.05

# terminar luego de 180 segundos
m.Params.TimeLimit = 180

Finalmente, resolvemos el modelo y mostramos la solución, en caso de que se haya encontrado una solución factible (si `m.SolCount` es estrictamente mayor a cero).

In [ ]:
# Se recomienda llamar a update() antes de resolver un modelo que usa restricciones lazy
m.update()

# calcular la solución óptima
m.optimize(mycallback)

# escribir la solución
if m.SolCount > 0:
    # recuperar los valores de las variables
    vx = m.getAttr('x', x)
    print('\nTour óptimo:')
    for i,j in A:
        if vx[i,j] >= 0.9:
            print('{} --> {}'.format(i, j))


Si encontramos alguna solución factible, el siguiente fragmento de código nos permite graficarla.

In [ ]:
# crear lista con arcos seleccionados en la solución
vx = m.getAttr('x', x)
L = gp.tuplelist([(i,j) for i,j in A if vx[i,j]>0.1])
# print(L)

# recuperar el tour como un ordenamiento de los nodos
T = [1]
# nodo actual:
i = 1
while True:
    # determinar sucesor de i
    a = L.select(i,'*')[0]
    L.remove(a)
    # colocar sucesor en la lista del tour y actualizar i
    T.append(a[1])
    i = a[1]
    # terminar cuando el nodo colocado sea 1
    if i==1: 
        break;
        
# Graficar el tour
dibujarTour(coordx, coordy, T)    

### Limitaciones:

Como los planos cortantes se generan solamente cuando hay una solución entera, este método no consigue actualizar con suficiente frecuencia las cotas inferiores y la exploración del árbol de branch-and-bound se torna ineficiente. Necesitamos un método que nos permita separar restricciones de corte a partir de una solución fraccionaria, para poder aplicarlo en cada nodo del árbol de branch-and-bound.

El problema de separar restricciones de corte a partir de una solución fraccionaria no es tan sencillo, pero puede reducirse a un problema de optimización combinatoria conocido: encontrar un corte de capacidad mínima en un grafo. El siguiente ejemplo ilustra una idea.

In [ ]:
import networkx as nx
import ipycytoscape

# Ejemplo de solución fraccionaria del TSP
# Nodos del grafo
V = gp.tuplelist(range(1,7))

# Arcos (grafo completo)
A = [(i,j) for i in V for j in V if i!=j]

# Valores de las variables en la solución fraccionaria
A1 = [(1,2), (3,1), (6,4), (5,6)]
A2 = [(2,3), (2,5), (4,3), (4,5)]
A3 = [(i,j) for (i,j) in A if (i,j) not in A1+A2 and (j,i) not in A1+A2]
A = A1+A2+A3
x={(i,j) : 1 if (i,j) in A1 else (0.5 if (i,j) in A2 else 0) for (i,j) in A}

D = nx.DiGraph()
D.add_nodes_from(V)
for i in V:
    D.nodes[i]['etiq']= str(i)
D.add_edges_from(A)
for i,j in A:
    D.edges[i,j]['etiq'] = str(x[i,j]) if x[i,j]>0.1 else ''
    D.edges[i,j]['color'] =  '#0000ff' if x[i,j]>=0.9 else ('#ff007f' if x[i,j]>=0.4 else '#e0e0e0')
grafo = ipycytoscape.CytoscapeWidget()
grafo.graph.add_graph_from_networkx(D, directed=True)
grafo.set_style([{'selector': 'node', 'style' : {'width': 15, 'height' : 15, 'background-color': '#11479e', 'font-family': 'helvetica', 'font-size': '10px', 'color':'white', 'label': 'data(etiq)', 'text-wrap' : 'wrap', 'text-valign' : 'center'}}, 
                    {'selector': 'node:parent', 'css': {'background-opacity': 0.333}, 'style' : {'font-family': 'helvetica', 'font-size': '10px', 'label': 'data(etiq)'}}, 
                    {'selector': 'edge', 'style': {'width': 1, 'line-color': 'data(color)', 'font-size': '10px', 'label': 'data(etiq)', 'text-valign' : 'top', 'text-margin-y' : '-10px'}}, 
                    {'selector': 'edge.directed', 'style': {'curve-style': 'bezier', 'target-arrow-shape': 'triangle', 'target-arrow-color': 'data(color)'}}])
grafo

## Código completo

Se reproduce a continuación el código completo del modelo anterior.

In [ ]:
# Implementación de modelos lineales enteros
# Problema del agente viajero asimétrico (ATSP)
# Implementación con desigualdades lazy

# Luis M. Torres (EPN 2022)

import gurobipy as gp
from gurobipy import GRB
import random as rm
import matplotlib.pyplot as plt

def dibujarTour(coordx, coordy, tour):
    Tx = [coordx[i] for i in tour]
    Ty = [coordy[i] for i in tour]
    plt.plot(Tx[:-1], Ty[:-1], 'ro')
    for i in range(len(tour)-1):
        s='{}'.format(tour[i])
        plt.text(Tx[i],Ty[i]+1,s)
        plt.arrow(Tx[i], Ty[i], Tx[i+1]-Tx[i], Ty[i+1]-Ty[i], color='blue', 
                  length_includes_head=True, width=0.1, head_width=2)
    display(plt.show())

# función callback para la separación de restricciones de corte
def mycallback(model, where):
    # esta función es llamada cuando se encuentra una nueva solución entera
    if where == GRB.Callback.MIPSOL:
        # recuperar los valores de la solución actual
        vx = model.cbGetSolution(model._x)
        # determinar los arcos seleccionados en la solución
        L = gp.tuplelist([(i,j) for (i,j) in model._A if vx[i,j]>=0.9])
        # construir la lista W de nodos que pueden ser alcanzados desde 1
        W = [1]
        i = 1
        while True :
            # seleccionar el único arco saliente de i en L
            a = L.select(i,'*')[0]
            L.remove(a)
            if a[1]==1: 
                break
            W.append(a[1])
            i = a[1]
        # si W!=V, agregar la desigualdad de corte asociada a W
        if len(W)!=len(model._V):
            Wc = [i for i in model._V if i not in W]
            model.cbLazy(model._x.sum(W, Wc) >= 1)
    
# número de nodos del grafo
n = 20

# conjunto de nodos del grafo
V = gp.tuplelist(range(1,n+1))

# posiciones de los nodos en un plano euclideano entre (0,0) y (100, 100)
coordx={i : rm.randint(0,100) for i in V}
coordy={i : rm.randint(0,100) for i in V}

# los costos son las distancias eculideanas
c = gp.tupledict({
    (i,j) : math.sqrt((coordx[i] - coordx[j])**2 + (coordy[i] - coordy[j])**2)
    for i in V for j in V if i!=j
})
A = c.keys()

try:
    # crear el objeto modelo
    m = gp.Model('atsp-corte-lazy')

    # crear las variables
    x = m.addVars(A, name="x", vtype=GRB.BINARY)

    # crear la función objetivo
    m.setObjective(x.prod(c,'*'), GRB.MINIMIZE)

    # copiar variables en el objeto modelo para tener acceso a x, V y A
    m._x = x
    m._V = V
    m._A = A

    # restricciones de grado saliente
    m.addConstrs((x.sum(i,'*') == 1 for i in V), "g_saliente")

    # restricciones de grado entrante
    m.addConstrs((x.sum('*', i) == 1 for i in V), "g_entrante")

    # Eecribir el modelo a un archivo
    # m.write('atsp-corte-lazy.lp')

    # configurar Gurobi para usar restricciones lazy
    m.Params.LazyConstraints = 1

    # terminar al alcanzar un Gap del 10%
    m.Params.MIPGap = 0.1

    # terminar luego de 180 segundos
    m.Params.TimeLimit = 180

    # se recomienda llamar a update() antes de resolver un modelo que usa restricciones lazy
    m.update()

    # resolver el modelo
    m.optimize(mycallback)

    # escribir la solución, si se encontró alguna
    if m.SolCount > 0:      
        # recuperar los valores de las variables
        vx = m.getAttr('x', x)
        print('\nTour óptimo:')
        for i,j in A:
            if vx[i,j] > 0:
                print('{} --> {}'.format(i, j))

        # recuperar el tour como un ordenamiento de los nodos
        L = gp.tuplelist([(i,j) for i,j in A if vx[i,j]>=0.9])
        T = [1]
        i = 1
        while True:
            # determinar sucesor de i
            a = L.select(i,'*')[0]
            L.remove(a)
            # colocar sucesor en la lista del tour y actualizar i
            T.append(a[1])
            i = a[1]
            # terminar cuando el nodo colocado sea 1
            if i==1: 
                break;
        # graficar el tour
        dibujarTour(coordx, coordy, T)        
        
except GurobiError as e:
    print('Se produjo un error de Gurobi: codigo: ' + str(e.errno) + ": " + str(e))

except AttributeError:
    print('Se produjo un error de atributo')